# Nuisance Alarm Filter

## Documentation

**General Logic**
1. Load cleaned data
2. Sort data by ASSET_ID_RAW + SCS_Time
3. Filter only for LOG_TYPE == "AL" (alarm events only)
4. Load the past 6 values (inclusive of current) of a concantenated EVENT_DESCRIPTION + EVENT_STATUS of the same asset (ASSET_ID_RAW) over a span of 1 min
5. Tag cases with consecutive repeated values as a nuisance alarm
6. Tag cases with alternating repeat values as a nuisance alarm

**Nuisance Alarm Logic**
1. Check 1st value n, assign it as value0
2. Check 2nd value n-1, assign it as value1
3. If value0 == value1:
       Tag event as a repeat nuisance alarm
4. Elif (value n-2 == value0) AND (value n-3 == value1) AND (value n-4 == value0) AND (value n-5 == value1):
       Tag event as repeat nuisance alarm
5.Else:
       No tag assigned




## Initialisation

### Load libraries

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import datetime as dt
import time
import pyxlsb

### Configure display

In [2]:
# Enable display of all columns for dataframes with many variables
pd.set_option('display.max_columns', None)

### Set Up Directory

In [3]:
# Check current directory location
import os
cwd = os.getcwd()
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\scripts'

In [4]:
# Define root file directory folder where the files are being stored
os.chdir(os.path.dirname(os.getcwd()) + '\\alarm-event-logs\\dataCleaned\\Run 003 - Sample from 27 Oct 2020')

# Check directory location
os.getcwd()

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\dataCleaned\\Run 003 - Sample from 27 Oct 2020'

In [5]:
# Get the list of all files in directory tree at given path
fileList = list()
for (dirpath, dirnames, filenames) in os.walk(os.getcwd()):
    #fileList += [os.path.join(dirpath, file) for file in filenames] # use this if you want to append full URL
    fileList += filenames
    
# Inspect data
fileList

['AE-cleaned-001 - Integrated.7z',
 'AE-cleaned-001 - Integrated.xlsb',
 'AE-cleaned-001.csv',
 'AL-cleaned-001.csv',
 'BI Dashboard - Run 003.pbix',
 'Nuisance Alarm Filter Output.csv',
 'Reference Index.xlsx']

### Load File

In [6]:
# Load data
df = pd.read_excel('AE-cleaned-001 - Integrated.xlsb', engine = 'pyxlsb', sheet_name='LeftMerge AE + AL')

# Inspect data structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186796 entries, 0 to 186795
Data columns (total 34 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ENTRY_CODE_SUFFIX         186796 non-null  object 
 1   ENTRY_CODE                186796 non-null  int64  
 2   ALARM_ID                  186796 non-null  int64  
 3   USER_ID                   186796 non-null  int64  
 4   EQUIPMENT_NAME            0 non-null       float64
 5   VALUE                     186796 non-null  object 
 6   VALUE_STATE               186796 non-null  int64  
 7   ACKNOWLEDGEMENT_REQUIRED  186796 non-null  int64  
 8   SEVERITY                  186796 non-null  int64  
 9   HIDDEN                    186796 non-null  int64  
 10  THEME                     186796 non-null  int64  
 11  EQUIPMENT_DATE            186796 non-null  float64
 12  ACQUISITION_DATE          186796 non-null  float64
 13  SCS_TIME                  186796 non-null  f

In [7]:
# Inspect data sample
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,LOG_TYPE,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE
0,+,-1139445989,14503,0,NaN,5,0,1,2,0,0,44131.101442,44131.101442,44131.101442,60,1,OCCCMS,1.0,OCC_LENV_CMS_,CMS SCS Server - Environment OCCCMS,Environment 1 Status,HOT,None,CMS SCS Server - Environment CMS,Environment Status,NaN,NaN,NaN,LENV,CMS,AL,44131.101443,44131.101443,44131.101443
1,+,-1139445990,14517,0,NaN,1,0,0,3,0,0,44131.101443,44131.101443,44131.101443,40,14,OCCCMS,1.0,COM/SER/B1/CAM20,SER:208 LWY1,Status,FAILURE,None,LWY,Status,NaN,NaN,NaN,COM,CAM,NaN,44131.101444,44131.101444,44131.101444
2,+,-1139446006,14533,0,NaN,1,1,0,4,0,0,44131.101473,44131.101473,44131.101473,44,10,OCCCMS,1.0,COM/FRP/1/PID16,Ent8:LD38,PID Status,COMM. LINK ERROR,None,Ent:LD,PID Status,NaN,NaN,NaN,COM,PID,AL,44131.101479,44131.101479,44131.101479
3,+,-1139445991,14518,0,NaN,0,0,0,2,0,0,44131.101443,44131.101443,44131.101443,48,11,OCCCMS,1.0,COM/BNK/B1/PABX01,PABX,Fan 1 Status,FAILURE,None,PABX,Fan Status,NaN,NaN,NaN,COM,PABX,NaN,44131.101444,44131.101444,44131.101444
4,+,-1139446221,2883,0,NaN,1,1,0,4,0,0,44131.102142,44131.102142,44131.102142,44,10,OCCCMS,1.0,COM/FRP/1/PID09,Ent1:LD31,PID Status,COMM. LINK ERROR,None,Ent:LD,PID Status,NaN,NaN,NaN,COM,PID,AL,44131.102150,44131.102150,44131.102150


In [8]:
# Format data (Convert Datetime format from Excel to Standard Unix Format)
df['DATETIME_SENT'] = pd.to_datetime(df['DATETIME_SENT'], unit='D',
               origin=pd.Timestamp('1900-01-01'))
df['DATETIME_RECEIVED'] = pd.to_datetime(df['DATETIME_RECEIVED'], unit='D',
               origin=pd.Timestamp('1900-01-01'))
df['EQUIPMENT_DATE'] = pd.to_datetime(df['EQUIPMENT_DATE'], unit='D',
               origin=pd.Timestamp('1900-01-01'))
df['ACQUISITION_DATE'] = pd.to_datetime(df['ACQUISITION_DATE'], unit='D',
               origin=pd.Timestamp('1900-01-01'))
df['SCS_TIME'] = pd.to_datetime(df['SCS_TIME'], unit='D',
               origin=pd.Timestamp('1900-01-01'))
df['TIME_CODE'] = pd.to_datetime(df['TIME_CODE'], unit='D',
               origin=pd.Timestamp('1900-01-01'))

# Inspect data structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186796 entries, 0 to 186795
Data columns (total 34 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ENTRY_CODE_SUFFIX         186796 non-null  object        
 1   ENTRY_CODE                186796 non-null  int64         
 2   ALARM_ID                  186796 non-null  int64         
 3   USER_ID                   186796 non-null  int64         
 4   EQUIPMENT_NAME            0 non-null       float64       
 5   VALUE                     186796 non-null  object        
 6   VALUE_STATE               186796 non-null  int64         
 7   ACKNOWLEDGEMENT_REQUIRED  186796 non-null  int64         
 8   SEVERITY                  186796 non-null  int64         
 9   HIDDEN                    186796 non-null  int64         
 10  THEME                     186796 non-null  int64         
 11  EQUIPMENT_DATE            186796 non-null  datetime64[ns]
 12  AC

In [9]:
# Inspect data sample
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,LOG_TYPE,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE
0,+,-1139445989,14503,0,NaN,5,0,1,2,0,0,2020-10-29 02:26:04.620000000,2020-10-29 02:26:04.620000000,2020-10-29 02:26:04.620000000,60,1,OCCCMS,1.0,OCC_LENV_CMS_,CMS SCS Server - Environment OCCCMS,Environment 1 Status,HOT,None,CMS SCS Server - Environment CMS,Environment Status,NaN,NaN,NaN,LENV,CMS,AL,2020-10-29 02:26:04.639000320,2020-10-29 02:26:04.639000320,2020-10-29 02:26:04.639999744
1,+,-1139445990,14517,0,NaN,1,0,0,3,0,0,2020-10-29 02:26:04.701000192,2020-10-29 02:26:04.701000192,2020-10-29 02:26:04.701000192,40,14,OCCCMS,1.0,COM/SER/B1/CAM20,SER:208 LWY1,Status,FAILURE,None,LWY,Status,NaN,NaN,NaN,COM,CAM,NaN,2020-10-29 02:26:04.753999616,2020-10-29 02:26:04.753999616,2020-10-29 02:26:04.753999616
2,+,-1139446006,14533,0,NaN,1,1,0,4,0,0,2020-10-29 02:26:07.256000256,2020-10-29 02:26:07.256000256,2020-10-29 02:26:07.256000256,44,10,OCCCMS,1.0,COM/FRP/1/PID16,Ent8:LD38,PID Status,COMM. LINK ERROR,None,Ent:LD,PID Status,NaN,NaN,NaN,COM,PID,AL,2020-10-29 02:26:07.753000192,2020-10-29 02:26:07.753000192,2020-10-29 02:26:07.753000192
3,+,-1139445991,14518,0,NaN,0,0,0,2,0,0,2020-10-29 02:26:04.705000192,2020-10-29 02:26:04.705000192,2020-10-29 02:26:04.705000192,48,11,OCCCMS,1.0,COM/BNK/B1/PABX01,PABX,Fan 1 Status,FAILURE,None,PABX,Fan Status,NaN,NaN,NaN,COM,PABX,NaN,2020-10-29 02:26:04.753999616,2020-10-29 02:26:04.753999616,2020-10-29 02:26:04.753999616
4,+,-1139446221,2883,0,NaN,1,1,0,4,0,0,2020-10-29 02:27:05.110999808,2020-10-29 02:27:05.110999808,2020-10-29 02:27:05.110999808,44,10,OCCCMS,1.0,COM/FRP/1/PID09,Ent1:LD31,PID Status,COMM. LINK ERROR,None,Ent:LD,PID Status,NaN,NaN,NaN,COM,PID,AL,2020-10-29 02:27:05.799000320,2020-10-29 02:27:05.799000320,2020-10-29 02:27:05.799000320


### Reindex Data By Time

In [10]:
# Create copy of time sequence field (SCS_TIME) as a proto index column
df["datetimeIndex"] = df["SCS_TIME"].copy()

# Set Date-Time to be the index
df = df.set_index(df["datetimeIndex"])

# Drop proto index column
del df["datetimeIndex"]

# Inspect data sample
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,LOG_TYPE,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE
datetimeIndex,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-10-29 02:26:04.620000000,+,-1139445989,14503,0,NaN,5,0,1,2,0,0,2020-10-29 02:26:04.620000000,2020-10-29 02:26:04.620000000,2020-10-29 02:26:04.620000000,60,1,OCCCMS,1.0,OCC_LENV_CMS_,CMS SCS Server - Environment OCCCMS,Environment 1 Status,HOT,None,CMS SCS Server - Environment CMS,Environment Status,NaN,NaN,NaN,LENV,CMS,AL,2020-10-29 02:26:04.639000320,2020-10-29 02:26:04.639000320,2020-10-29 02:26:04.639999744
2020-10-29 02:26:04.701000192,+,-1139445990,14517,0,NaN,1,0,0,3,0,0,2020-10-29 02:26:04.701000192,2020-10-29 02:26:04.701000192,2020-10-29 02:26:04.701000192,40,14,OCCCMS,1.0,COM/SER/B1/CAM20,SER:208 LWY1,Status,FAILURE,None,LWY,Status,NaN,NaN,NaN,COM,CAM,NaN,2020-10-29 02:26:04.753999616,2020-10-29 02:26:04.753999616,2020-10-29 02:26:04.753999616
2020-10-29 02:26:07.256000256,+,-1139446006,14533,0,NaN,1,1,0,4,0,0,2020-10-29 02:26:07.256000256,2020-10-29 02:26:07.256000256,2020-10-29 02:26:07.256000256,44,10,OCCCMS,1.0,COM/FRP/1/PID16,Ent8:LD38,PID Status,COMM. LINK ERROR,None,Ent:LD,PID Status,NaN,NaN,NaN,COM,PID,AL,2020-10-29 02:26:07.753000192,2020-10-29 02:26:07.753000192,2020-10-29 02:26:07.753000192
2020-10-29 02:26:04.705000192,+,-1139445991,14518,0,NaN,0,0,0,2,0,0,2020-10-29 02:26:04.705000192,2020-10-29 02:26:04.705000192,2020-10-29 02:26:04.705000192,48,11,OCCCMS,1.0,COM/BNK/B1/PABX01,PABX,Fan 1 Status,FAILURE,None,PABX,Fan Status,NaN,NaN,NaN,COM,PABX,NaN,2020-10-29 02:26:04.753999616,2020-10-29 02:26:04.753999616,2020-10-29 02:26:04.753999616
2020-10-29 02:27:05.110999808,+,-1139446221,2883,0,NaN,1,1,0,4,0,0,2020-10-29 02:27:05.110999808,2020-10-29 02:27:05.110999808,2020-10-29 02:27:05.110999808,44,10,OCCCMS,1.0,COM/FRP/1/PID09,Ent1:LD31,PID Status,COMM. LINK ERROR,None,Ent:LD,PID Status,NaN,NaN,NaN,COM,PID,AL,2020-10-29 02:27:05.799000320,2020-10-29 02:27:05.799000320,2020-10-29 02:27:05.799000320


In [11]:
# Sort Data By ASSETNAME and DT (DEFAULT)
df = df.sort_index().sort_values(["ENVIRONMENT", "ASSET_ID_RAW", "ALARM_ID", "SCS_TIME"])

# Inspect data sample
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,LOG_TYPE,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE
datetimeIndex,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-10-29 15:17:32.509999616,+,-1139595636,160281,0,NaN,1,0,0,3,0,0,2020-10-29 15:17:32.509999616,2020-10-29 15:17:32.509999616,2020-10-29 15:17:32.509999616,40,14,OCCCMS,1.0,CAM/SER/1/CAM34,SER:306 E3ES0708,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 15:17:32.755000064,2020-10-29 15:17:32.755000064,2020-10-29 15:17:32.755000064
2020-10-29 15:17:33.511000064,+,-1139595638,160283,0,NaN,0,0,0,3,0,0,2020-10-29 15:17:33.511000064,2020-10-29 15:17:33.511000064,2020-10-29 15:17:33.511000064,40,14,OCCCMS,1.0,CAM/SER/1/CAM34,SER:306 E3ES0708,Status,NORMAL,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 15:17:33.752999936,2020-10-29 15:17:33.752999936,2020-10-29 15:17:33.754000128
2020-10-29 02:31:08.972999936,+,-1139447292,15771,0,NaN,1,0,0,3,0,0,2020-10-29 02:31:08.972999936,2020-10-29 02:31:08.972999936,2020-10-29 02:31:08.972999936,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.752000256
2020-10-29 02:31:09.479000320,+,-1139447295,15774,0,NaN,0,0,0,3,0,0,2020-10-29 02:31:09.479000320,2020-10-29 02:31:09.479000320,2020-10-29 02:31:09.479000320,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,NORMAL,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.752000256
2020-10-29 03:21:42.381999872,+,-1139458206,26181,0,NaN,1,0,0,3,0,0,2020-10-29 03:21:42.381999872,2020-10-29 03:21:42.381999872,2020-10-29 03:21:42.381999872,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 03:21:42.755000064,2020-10-29 03:21:42.755000064,2020-10-29 03:21:42.755000064


## Nuisance Alarm Suppression

In [12]:
# Concat Event Description + Status values
df["eventValue_N"] = df["EVENT_DESCRIPTION"] + " | " + df["EVENT_STATUS"]

# Inspect data sample
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,LOG_TYPE,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,eventValue_N
datetimeIndex,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-10-29 15:17:32.509999616,+,-1139595636,160281,0,NaN,1,0,0,3,0,0,2020-10-29 15:17:32.509999616,2020-10-29 15:17:32.509999616,2020-10-29 15:17:32.509999616,40,14,OCCCMS,1.0,CAM/SER/1/CAM34,SER:306 E3ES0708,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 15:17:32.755000064,2020-10-29 15:17:32.755000064,2020-10-29 15:17:32.755000064,Status | FAILURE
2020-10-29 15:17:33.511000064,+,-1139595638,160283,0,NaN,0,0,0,3,0,0,2020-10-29 15:17:33.511000064,2020-10-29 15:17:33.511000064,2020-10-29 15:17:33.511000064,40,14,OCCCMS,1.0,CAM/SER/1/CAM34,SER:306 E3ES0708,Status,NORMAL,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 15:17:33.752999936,2020-10-29 15:17:33.752999936,2020-10-29 15:17:33.754000128,Status | NORMAL
2020-10-29 02:31:08.972999936,+,-1139447292,15771,0,NaN,1,0,0,3,0,0,2020-10-29 02:31:08.972999936,2020-10-29 02:31:08.972999936,2020-10-29 02:31:08.972999936,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.752000256,Status | FAILURE
2020-10-29 02:31:09.479000320,+,-1139447295,15774,0,NaN,0,0,0,3,0,0,2020-10-29 02:31:09.479000320,2020-10-29 02:31:09.479000320,2020-10-29 02:31:09.479000320,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,NORMAL,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.752000256,Status | NORMAL
2020-10-29 03:21:42.381999872,+,-1139458206,26181,0,NaN,1,0,0,3,0,0,2020-10-29 03:21:42.381999872,2020-10-29 03:21:42.381999872,2020-10-29 03:21:42.381999872,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 03:21:42.755000064,2020-10-29 03:21:42.755000064,2020-10-29 03:21:42.755000064,Status | FAILURE


In [13]:
# Past 5 values for the same ["ENVIRONMENT", "ASSET_ID_RAW", "ALARM_ID"] within in a 1 min window
df["eventValue_N-1"] = df.groupby(["ENVIRONMENT", "ASSET_ID_RAW", "ALARM_ID"])["eventValue_N"].shift(1)
df["eventValue_N-2"] = df.groupby(["ENVIRONMENT", "ASSET_ID_RAW", "ALARM_ID"])["eventValue_N"].shift(2)
df["eventValue_N-3"] = df.groupby(["ENVIRONMENT", "ASSET_ID_RAW", "ALARM_ID"])["eventValue_N"].shift(3)
df["eventValue_N-4"] = df.groupby(["ENVIRONMENT", "ASSET_ID_RAW", "ALARM_ID"])["eventValue_N"].shift(4)
df["eventValue_N-5"] = df.groupby(["ENVIRONMENT", "ASSET_ID_RAW", "ALARM_ID"])["eventValue_N"].shift(5)

# Get Timestamps of N-1 and N-5
df["SCS_TIME_N-1"] = df.groupby(["ENVIRONMENT", "ASSET_ID_RAW", "ALARM_ID"])["SCS_TIME"].shift(1)
df["SCS_TIME_N-3"] = df.groupby(["ENVIRONMENT", "ASSET_ID_RAW", "ALARM_ID"])["SCS_TIME"].shift(3)
df["SCS_TIME_N-5"] = df.groupby(["ENVIRONMENT", "ASSET_ID_RAW", "ALARM_ID"])["SCS_TIME"].shift(5)

# Inspect data sample
#df.head()
df.loc[~df["eventValue_N-1"].isnull()].head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,LOG_TYPE,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,eventValue_N,eventValue_N-1,eventValue_N-2,eventValue_N-3,eventValue_N-4,eventValue_N-5,SCS_TIME_N-1,SCS_TIME_N-3,SCS_TIME_N-5
datetimeIndex,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-10-29 06:31:19.573000192,+,-1139499507,32029,0,NaN,1,0,0,3,0,0,2020-10-29 06:31:19.573000192,2020-10-29 06:31:19.573000192,2020-10-29 06:31:19.573000192,34,6,OCCCMS,1.0,CIV/CNT/1/RSH06,Roller Shutter for Entrance 5,Position,UP,None,Roller Shutter for SUBLOCATION,Position,NaN,NaN,NaN,CIV,RSH,NaN,2020-10-29 06:31:19.752999936,2020-10-29 06:31:19.752999936,2020-10-29 06:31:19.754000128,Position | UP,Position | DOWN,NaN,NaN,NaN,NaN,2020-10-29 03:49:50.153000192,NaT,NaT
2020-10-29 04:36:05.241000192,+,-1139474017,22656,0,NaN,1,0,0,3,0,0,2020-10-29 04:36:05.241000192,2020-10-29 04:36:05.241000192,2020-10-29 04:36:05.241000192,34,4,OCCCMS,1.0,CIV/HBF/1/RSH07,Roller Shutter for Entrance 3,Position,UP,None,Roller Shutter for SUBLOCATION,Position,NaN,NaN,NaN,CIV,RSH,NaN,2020-10-29 04:36:05.757000192,2020-10-29 04:36:05.757000192,2020-10-29 04:36:05.758000384,Position | UP,Position | DOWN,NaN,NaN,NaN,NaN,2020-10-29 03:05:15.646000384,NaT,NaT
2020-10-29 06:11:22.924000256,+,-1139495408,41452,0,NaN,1,0,0,3,0,0,2020-10-29 06:11:22.924000256,2020-10-29 06:11:22.924000256,2020-10-29 06:11:22.924000256,34,4,OCCCMS,1.0,CIV/HBF/1/RSH07,Roller Shutter for Entrance 3,Position,UP,NaN,Roller Shutter for SUBLOCATION,Position,NaN,NaN,NaN,CIV,RSH,NaN,2020-10-29 06:11:23.756000000,2020-10-29 06:11:23.756000000,2020-10-29 06:11:23.757000192,Position | UP,Position | DOWN,NaN,NaN,NaN,NaN,2020-10-29 04:36:09.277999872,NaT,NaT
2020-10-29 05:15:34.357999872,+,-1139482727,49724,0,NaN,1,0,0,3,0,0,2020-10-29 05:15:34.357999872,2020-10-29 05:15:34.357999872,2020-10-29 05:15:34.357999872,34,15,OCCCMS,1.0,CIV/KVN/B1/FSH01,Fusible Link Fire Shutter for Dirty Area 1,Position,OPEN,None,Fusible Link Fire Shutter for SUBLOCATION,Position,NaN,NaN,NaN,CIV,FSH,NaN,2020-10-29 05:15:34.754999808,2020-10-29 05:15:34.754999808,2020-10-29 05:15:34.754999808,Position | OPEN,Position | CLOSE,NaN,NaN,NaN,NaN,2020-10-29 05:15:31.837999872,NaT,NaT
2020-10-29 05:17:14.980999936,+,-1139483143,50138,0,NaN,1,0,0,3,0,0,2020-10-29 05:17:14.980999936,2020-10-29 05:17:14.980999936,2020-10-29 05:17:14.980999936,34,15,OCCCMS,1.0,CIV/KVN/B1/FSH01,Fusible Link Fire Shutter for Dirty Area 1,Position,OPEN,None,Fusible Link Fire Shutter for SUBLOCATION,Position,NaN,NaN,NaN,CIV,FSH,NaN,2020-10-29 05:17:15.755000064,2020-10-29 05:17:15.755000064,2020-10-29 05:17:15.755000064,Position | OPEN,Position | CLOSE,NaN,NaN,NaN,NaN,2020-10-29 05:17:13.463000064,NaT,NaT


In [14]:
# Reset index to serial number
df = df.reset_index()

# Drop former index column
df = df.drop(columns = ["datetimeIndex"])

# Inspect data sample
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,LOG_TYPE,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,eventValue_N,eventValue_N-1,eventValue_N-2,eventValue_N-3,eventValue_N-4,eventValue_N-5,SCS_TIME_N-1,SCS_TIME_N-3,SCS_TIME_N-5
0,+,-1139595636,160281,0,NaN,1,0,0,3,0,0,2020-10-29 15:17:32.509999616,2020-10-29 15:17:32.509999616,2020-10-29 15:17:32.509999616,40,14,OCCCMS,1.0,CAM/SER/1/CAM34,SER:306 E3ES0708,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 15:17:32.755000064,2020-10-29 15:17:32.755000064,2020-10-29 15:17:32.755000064,Status | FAILURE,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
1,+,-1139595638,160283,0,NaN,0,0,0,3,0,0,2020-10-29 15:17:33.511000064,2020-10-29 15:17:33.511000064,2020-10-29 15:17:33.511000064,40,14,OCCCMS,1.0,CAM/SER/1/CAM34,SER:306 E3ES0708,Status,NORMAL,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 15:17:33.752999936,2020-10-29 15:17:33.752999936,2020-10-29 15:17:33.754000128,Status | NORMAL,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
2,+,-1139447292,15771,0,NaN,1,0,0,3,0,0,2020-10-29 02:31:08.972999936,2020-10-29 02:31:08.972999936,2020-10-29 02:31:08.972999936,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.752000256,Status | FAILURE,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
3,+,-1139447295,15774,0,NaN,0,0,0,3,0,0,2020-10-29 02:31:09.479000320,2020-10-29 02:31:09.479000320,2020-10-29 02:31:09.479000320,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,NORMAL,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.752000256,Status | NORMAL,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
4,+,-1139458206,26181,0,NaN,1,0,0,3,0,0,2020-10-29 03:21:42.381999872,2020-10-29 03:21:42.381999872,2020-10-29 03:21:42.381999872,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 03:21:42.755000064,2020-10-29 03:21:42.755000064,2020-10-29 03:21:42.755000064,Status | FAILURE,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT


In [15]:
# Initialise Field with an Empty Column
#df.reindex(columns = df.columns.tolist() + ["NuisanceAlarm"])
df["NuisanceAlarm"] = False
df["RepeatAlarm"] = False
df["AltAlarm2"] = False
df["AltAlarm3"] = False

# Check for consecutive repeat alarms
df.loc[(~df["eventValue_N-1"].isnull()) & # ensure that there is a value to compare to
       (df["SCS_TIME_N-1"] <= (df["SCS_TIME"] + pd.Timedelta(seconds=60))) & # check if timeframe is within 60s
       (df["eventValue_N"] == df["eventValue_N-1"]) # compare values for repeat values
      , "NuisanceAlarm"] = True

# Check for toggle alarms (3 Alternating Pairs)
df.loc[(~df["eventValue_N-3"].isnull()) & # ensure that there is a value to compare to
       (df["SCS_TIME_N-3"] <= (df["SCS_TIME"] + pd.Timedelta(seconds=60))) & # check if timeframe is within 60s
       (df["eventValue_N"] != df["eventValue_N-1"]) & # ensure the eventValue_N (A) and eventValue_N-1 (B) are different
       (df["eventValue_N"] == df["eventValue_N-2"]) & # check for alternating pattern A1
       (df["eventValue_N-1"] == df["eventValue_N-3"]) # check for alternating pattern B1
      , "NuisanceAlarm"] = True

# Additional Code for Testing

# Check for consecutive repeat alarms
df.loc[(~df["eventValue_N-1"].isnull()) & # ensure that there is a value to compare to
       (df["SCS_TIME_N-1"] <= (df["SCS_TIME"] + pd.Timedelta(seconds=60))) & # check if timeframe is within 60s
       (df["eventValue_N"] == df["eventValue_N-1"]) # compare values for repeat values
      , "RepeatAlarm"] = True

# Check for toggle alarms (3 Alternating Pairs)
df.loc[(~df["eventValue_N-3"].isnull()) & # ensure that there is a value to compare to
       (df["SCS_TIME_N-3"] <= (df["SCS_TIME"] + pd.Timedelta(seconds=60))) & # check if timeframe is within 60s
       (df["eventValue_N"] != df["eventValue_N-1"]) & # ensure the eventValue_N (A) and eventValue_N-1 (B) are different
       (df["eventValue_N"] == df["eventValue_N-2"]) & # check for alternating pattern A1
       (df["eventValue_N-1"] == df["eventValue_N-3"]) # check for alternating pattern B1
      , "AltAlarm2"] = True

# Check for toggle alarms (3 Alternating Pairs)
df.loc[(~df["eventValue_N-5"].isnull()) & # ensure that there is a value to compare to
       (df["SCS_TIME_N-5"] <= (df["SCS_TIME"] + pd.Timedelta(seconds=60))) & # check if timeframe is within 60s
       (df["eventValue_N"] != df["eventValue_N-1"]) & # ensure the eventValue_N (A) and eventValue_N-1 (B) are different
       (df["eventValue_N"] == df["eventValue_N-2"]) & # check for alternating pattern A1
       (df["eventValue_N-1"] == df["eventValue_N-3"]) & # check for alternating pattern B1
       (df["eventValue_N"] == df["eventValue_N-4"]) & # check for alternating pattern A2
       (df["eventValue_N-1"] == df["eventValue_N-5"]) # check for alternating pattern B2
      , "AltAlarm3"] = True

# Inspect data sample
df.head()
#df.loc[df["NuisanceAlarm"] == True].head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,LOG_TYPE,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,eventValue_N,eventValue_N-1,eventValue_N-2,eventValue_N-3,eventValue_N-4,eventValue_N-5,SCS_TIME_N-1,SCS_TIME_N-3,SCS_TIME_N-5,NuisanceAlarm,RepeatAlarm,AltAlarm2,AltAlarm3
0,+,-1139595636,160281,0,NaN,1,0,0,3,0,0,2020-10-29 15:17:32.509999616,2020-10-29 15:17:32.509999616,2020-10-29 15:17:32.509999616,40,14,OCCCMS,1.0,CAM/SER/1/CAM34,SER:306 E3ES0708,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 15:17:32.755000064,2020-10-29 15:17:32.755000064,2020-10-29 15:17:32.755000064,Status | FAILURE,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,False,False,False,False
1,+,-1139595638,160283,0,NaN,0,0,0,3,0,0,2020-10-29 15:17:33.511000064,2020-10-29 15:17:33.511000064,2020-10-29 15:17:33.511000064,40,14,OCCCMS,1.0,CAM/SER/1/CAM34,SER:306 E3ES0708,Status,NORMAL,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 15:17:33.752999936,2020-10-29 15:17:33.752999936,2020-10-29 15:17:33.754000128,Status | NORMAL,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,False,False,False,False
2,+,-1139447292,15771,0,NaN,1,0,0,3,0,0,2020-10-29 02:31:08.972999936,2020-10-29 02:31:08.972999936,2020-10-29 02:31:08.972999936,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.752000256,Status | FAILURE,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,False,False,False,False
3,+,-1139447295,15774,0,NaN,0,0,0,3,0,0,2020-10-29 02:31:09.479000320,2020-10-29 02:31:09.479000320,2020-10-29 02:31:09.479000320,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,NORMAL,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.752000256,Status | NORMAL,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,False,False,False,False
4,+,-1139458206,26181,0,NaN,1,0,0,3,0,0,2020-10-29 03:21:42.381999872,2020-10-29 03:21:42.381999872,2020-10-29 03:21:42.381999872,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 03:21:42.755000064,2020-10-29 03:21:42.755000064,2020-10-29 03:21:42.755000064,Status | FAILURE,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,False,False,False,False


In [16]:
# Delete helper columns
df = df.drop(columns=["eventValue_N", 
                      "eventValue_N-1",
                      "eventValue_N-2",
                      "eventValue_N-3",
                      "eventValue_N-4",
                      "eventValue_N-5",
                      "SCS_TIME_N-1",
                      "SCS_TIME_N-3",
                      "SCS_TIME_N-5"])

# Inspect data sample
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,LOG_TYPE,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,NuisanceAlarm,RepeatAlarm,AltAlarm2,AltAlarm3
0,+,-1139595636,160281,0,NaN,1,0,0,3,0,0,2020-10-29 15:17:32.509999616,2020-10-29 15:17:32.509999616,2020-10-29 15:17:32.509999616,40,14,OCCCMS,1.0,CAM/SER/1/CAM34,SER:306 E3ES0708,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 15:17:32.755000064,2020-10-29 15:17:32.755000064,2020-10-29 15:17:32.755000064,False,False,False,False
1,+,-1139595638,160283,0,NaN,0,0,0,3,0,0,2020-10-29 15:17:33.511000064,2020-10-29 15:17:33.511000064,2020-10-29 15:17:33.511000064,40,14,OCCCMS,1.0,CAM/SER/1/CAM34,SER:306 E3ES0708,Status,NORMAL,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 15:17:33.752999936,2020-10-29 15:17:33.752999936,2020-10-29 15:17:33.754000128,False,False,False,False
2,+,-1139447292,15771,0,NaN,1,0,0,3,0,0,2020-10-29 02:31:08.972999936,2020-10-29 02:31:08.972999936,2020-10-29 02:31:08.972999936,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.752000256,False,False,False,False
3,+,-1139447295,15774,0,NaN,0,0,0,3,0,0,2020-10-29 02:31:09.479000320,2020-10-29 02:31:09.479000320,2020-10-29 02:31:09.479000320,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,NORMAL,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.751000064,2020-10-29 02:31:09.752000256,False,False,False,False
4,+,-1139458206,26181,0,NaN,1,0,0,3,0,0,2020-10-29 03:21:42.381999872,2020-10-29 03:21:42.381999872,2020-10-29 03:21:42.381999872,40,14,OCCCMS,1.0,CAM/SER/1/CAM35,SER:307 E4ES0102,Status,FAILURE,None,EES,Status,NaN,NaN,NaN,CAM,CAM,NaN,2020-10-29 03:21:42.755000064,2020-10-29 03:21:42.755000064,2020-10-29 03:21:42.755000064,False,False,False,False


In [17]:
# Inspect data structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186796 entries, 0 to 186795
Data columns (total 38 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ENTRY_CODE_SUFFIX         186796 non-null  object        
 1   ENTRY_CODE                186796 non-null  int64         
 2   ALARM_ID                  186796 non-null  int64         
 3   USER_ID                   186796 non-null  int64         
 4   EQUIPMENT_NAME            0 non-null       float64       
 5   VALUE                     186796 non-null  object        
 6   VALUE_STATE               186796 non-null  int64         
 7   ACKNOWLEDGEMENT_REQUIRED  186796 non-null  int64         
 8   SEVERITY                  186796 non-null  int64         
 9   HIDDEN                    186796 non-null  int64         
 10  THEME                     186796 non-null  int64         
 11  EQUIPMENT_DATE            186796 non-null  datetime64[ns]
 12  AC

In [18]:

print("Total: " + str(df["RepeatAlarm"].count()))
print("Total Alarms: " + str(df.loc[df["LOG_TYPE"] == "AL", "RepeatAlarm"].count()))
print("Total Alarms %: " + str(df.loc[df["LOG_TYPE"] == "AL", "RepeatAlarm"].count() / df["RepeatAlarm"].count() * 100))

print("Repeat Events: " + str(df.loc[df["RepeatAlarm"] == True, "RepeatAlarm"].count()))
print("Repeat Events %: " + str(df.loc[df["RepeatAlarm"] == True, "RepeatAlarm"].count() / df["RepeatAlarm"].count() * 100))
print("Repeat Alarm Events %: " + str(df.loc[df["RepeatAlarm"] == True, "RepeatAlarm"].count() / df.loc[df["LOG_TYPE"] == "AL", "RepeatAlarm"].count() * 100))

print("AltAlarm2 Events: " + str(df.loc[df["AltAlarm2"] == True, "RepeatAlarm"].count()))
print("AltAlarm2 Events %: " + str(df.loc[df["AltAlarm2"] == True, "RepeatAlarm"].count() / df["RepeatAlarm"].count() * 100))
print("AltAlarm2 Alarm Events %: " + str(df.loc[df["AltAlarm2"] == True, "RepeatAlarm"].count() / df.loc[df["LOG_TYPE"] == "AL", "RepeatAlarm"].count() * 100))

print("AltAlarm3 Events: " + str(df.loc[df["AltAlarm3"] == True, "RepeatAlarm"].count()))
print("AltAlarm3 Events %: " + str(df.loc[df["AltAlarm3"] == True, "RepeatAlarm"].count() / df["RepeatAlarm"].count() * 100))
print("AltAlarm3 Alarm Events %: " + str(df.loc[df["AltAlarm3"] == True, "RepeatAlarm"].count() / df.loc[df["LOG_TYPE"] == "AL", "RepeatAlarm"].count() * 100))


Total: 186796
Total Alarms: 2364
Total Alarms %: 1.2655517248763357
Repeat Events: 1049
Repeat Events %: 0.561575194329643
Repeat Alarm Events %: 44.373942470389174
AltAlarm2 Events: 724
AltAlarm2 Events %: 0.3875885993276087
AltAlarm2 Alarm Events %: 30.62605752961083
AltAlarm3 Events: 654
AltAlarm3 Events %: 0.3501145634810167
AltAlarm3 Alarm Events %: 27.66497461928934


In [19]:
# Delete test columns
df = df.drop(columns=["RepeatAlarm", 
                      "AltAlarm2",
                      "AltAlarm3"])

# Inspect data structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186796 entries, 0 to 186795
Data columns (total 35 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ENTRY_CODE_SUFFIX         186796 non-null  object        
 1   ENTRY_CODE                186796 non-null  int64         
 2   ALARM_ID                  186796 non-null  int64         
 3   USER_ID                   186796 non-null  int64         
 4   EQUIPMENT_NAME            0 non-null       float64       
 5   VALUE                     186796 non-null  object        
 6   VALUE_STATE               186796 non-null  int64         
 7   ACKNOWLEDGEMENT_REQUIRED  186796 non-null  int64         
 8   SEVERITY                  186796 non-null  int64         
 9   HIDDEN                    186796 non-null  int64         
 10  THEME                     186796 non-null  int64         
 11  EQUIPMENT_DATE            186796 non-null  datetime64[ns]
 12  AC

## Export File

In [20]:
# Save File
df.to_csv('Nuisance Alarm Filter Output' + '.csv', index=False)